## 01. 라이브러리 로드

In [10]:
import pandas as pd
import numpy as np

## 02. 데이터 로드 및 정제

In [11]:
df = pd.read_excel('data/krx/daily_close_2011_2024_04.xlsx')

#### 데이터 정제
- CAGR 계산에 사용할 컬럼 지정
- 컬럼에 결측치가 있거나 숫자로 변환할 수 없는 행 제거

In [12]:
COMPANY_COL = '거래소코드'       # 회사명 컬럼
YEAR_COL = '회계년도'       # 회계년도 컬럼
PRICE_COL = '수정종가'      # CAGR 계산에 사용할 가격 컬럼

essential_cols = [COMPANY_COL, YEAR_COL, PRICE_COL]
df.dropna(subset=essential_cols, inplace=True)
df[PRICE_COL] = pd.to_numeric(df[PRICE_COL], errors='coerce')
df[YEAR_COL] = pd.to_numeric(df[YEAR_COL], errors='coerce')
df.dropna(subset=[PRICE_COL, YEAR_COL], inplace=True)
df[YEAR_COL] = df[YEAR_COL].astype(int)

## 03. CAGR 계산
- 최종 결과를 위한 빈 리스트 생성
- 회사명 그룹을 묶어 각 회사에 대해 반복
    - 기준이 되는 시작 시점의 정보 고정
    - 두번쨰 해부터 마지막 해까지 각 해를 대상년도를 설정하여 반복
        - 기간 계산 (대상년도 - 시작년도)
        - CAGR 계산
        - 리스트 추가
- DataFrame 변환

In [13]:
results_list = []

# '회사명'으로 그룹을 묶어 각 회사에 대해 반복
for company_name, group in df.groupby(COMPANY_COL):

    group = group.sort_values(by=YEAR_COL)
    
    # 데이터가 2년 치 이상 있어야 계산이 가능
    if len(group) < 2:
        continue

    # 기준이 되는 시작 시점(첫 해)의 정보를 고정
    start_row = group.iloc[0]
    start_year = start_row[YEAR_COL]
    start_price = start_row[PRICE_COL]

    # 두 번째 해부터 마지막 해까지, 각 해를 '대상년도'로 설정하여 반복
    for i in range(1, len(group)):
        current_row = group.iloc[i]
        current_year = current_row[YEAR_COL]
        current_price = current_row[PRICE_COL]
        
        num_years = current_year - start_year
        
        # CAGR을 계산
        cagr = np.nan
        if num_years > 0 and start_price > 0:
            cagr = (current_price / start_price) ** (1 / num_years) - 1

        results_list.append({
            '거래소코드': company_name,
            '시작년도': start_year,
            '대상년도': current_year,
            'CAGR': cagr
        })

final_df = pd.DataFrame(results_list)

#### CAGR 컬럼 퍼센트(%)형식으로 변환

In [14]:
if not final_df.empty:
    final_df['CAGR'] = final_df['CAGR'].apply(
        # x에 100을 곱한 값을 소수점 둘째 자리에서 반올림
        lambda x: round(x * 100, 2) if pd.notna(x) else np.nan
    )

#### 시작년도 컬럼 삭제및 컬럼명 재정의

In [15]:
final_df.drop(columns=['시작년도'], inplace=True)
final_df.rename(columns={'대상년도': '회계년도'}, inplace=True)
final_df['회계년도'] = final_df['회계년도'] - 1

## 04. 데이터 저장

In [16]:
final_df.to_excel('data/krx/누적_CAGR.xlsx', index=False)